In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import qutip as qt 
from ECD_control.ECD_optimization.tf_adam_optimizer import AdamOptimizer
from ECD_control.gate_sets.GRAPE import GRAPE
from ECD_control.ECD_optimization.GateSynthesizer import GateSynthesizer
from ECD_control.ECD_optimization.optimization_analysis import OptimizationAnalysis, OptimizationSweepsAnalysis
import matplotlib.pyplot as plt
from tensorflow.python.client import device_lib
device_lib.list_local_devices()


Need tf version 2.3.0 or later. Using tensorflow version: 2.8.0


Need tf version 2.3.0 or later. Using tensorflow version: 2.8.0



[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 154751326718694639
 xla_global_id: -1,
 name: "/device:GPU:0"
 device_type: "GPU"
 memory_limit: 7806648320
 locality {
   bus_id: 1
   links {
   }
 }
 incarnation: 11234017601819634380
 physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080, pci bus id: 0000:0a:00.0, compute capability: 8.6"
 xla_global_id: 416903419]

In [2]:
q_dim = 4
c_dim = 30
DAC_time_resolution = 2
fock = 4

# In GHz = cycles / ns
anharm = .4
kerr = 1e-5
chi = 2e-3
drive = D = 2 * np.pi * 1e-3

a = qt.tensor(qt.destroy(c_dim), qt.qeye(q_dim))
b = qt.tensor(qt.qeye(c_dim), qt.destroy(q_dim))
ad = a.dag()
bd = b.dag()
H0 = (anharm/2) * bd * bd * b * b
H0 += (kerr/2) * ad * ad * a * a
H0 += (chi) * ad * a * bd * b
H0 *= 2*np.pi
Hcs = [D*(b + bd), 1j*D*(b - bd), D*(a + ad), 1j*D*(a - ad)]

init_states = [
    qt.tensor(qt.basis(c_dim, 0), qt.basis(q_dim, 0))
]

final_states = [
    qt.tensor(qt.basis(c_dim, fock), qt.basis(q_dim, 0))
]

In [3]:
# We initialize the gateset here
gate_set_params = {
    'H_static' : H0,
    'H_control' : Hcs,
    'DAC_delta_t' : DAC_time_resolution,
    'pulse_delta_t' : DAC_time_resolution,
    'scale' : 1.0 # range of DAC amplitudes for initial random waves
}
GRAPE_gate_set = GRAPE(**gate_set_params)

In [4]:
#the optimization options
synth_params = {
    'gateset' : GRAPE_gate_set,
    'N_blocks': 300, # note that the length of the pulse is this times the DAC_time_resolution
    'N_multistart' : 20, #Batch size (number of circuit optimizations to run in parallel)
    'epochs' : 1000, #number of epochs before termination
    'epoch_size' : 10, #number of adam steps per epoch
    'learning_rate' : 0.01, #adam learning rate
    'term_fid' : 0.995, #terminal fidelitiy
    'dfid_stop' : 1e-6, #stop if dfid between two epochs is smaller than this number
    'initial_states' : init_states, #qubit tensor oscillator, start in |g> |0>
    'target_states' : final_states, #end in |e> |target>.
    'name' : 'GRAPE Fock %d' % fock, #name for printing and saving
    'filename' : None, #if no filename specified, results will be saved in this folder under 'name.h5'
}
gatesynth = GateSynthesizer(**synth_params)

In [ ]:
#create optimization object. 
#initial params will be randomized upon creation
opt = AdamOptimizer(gatesynth)

#print optimization info. this lives in gatesynth, since we eventually want to fully abstract away the optimizer
gatesynth.best_fidelity()

In [ ]:
#run optimizer.
#note the optimizer can be stopped at any time by interrupting the python consle,
#and the optimization results will still be saved and part of the opt object.
#This allows you to stop the optimization whenever you want and still use the result.
opt.optimize()

In [ ]:
#can print info, including the best circuit found.
gatesynth.print_info() 

In [ ]:
print(gatesynth.best_fidelity())

In [ ]:
#can also get the best circuit parameters directly, could save this to a .npz file.
best_circuit =  gatesynth.best_circuit()
betas = best_circuit['betas']
thetas = best_circuit['thetas']
print(best_circuit)